In [ ]:
## @see https://www.kaggle.com/toorkp/churn-wsdm/data

In [4]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import pandas as pd
import time
import gc; gc.enable()
import time
import sys

from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

from collections import Counter
from numpy.random import RandomState
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

pd.options.display.float_format = "{:.2f}".format
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('modules')

from shared_functions import *

import pickle
from datetime import datetime

In [6]:
# Load the pre-split data

store = pd.HDFStore('/home/dissertation/data/base_abt_split.h5')
X_train, X_test, y_train, y_test = store['X_train'], store['X_test'], store['y_train'], store['y_test']
store.close()

print("Train Shape:", X_train.shape, y_train.shape)
print("Train Shape:", X_test.shape, y_test.shape)
X_train.head()

Train Shape: (600803, 16) (600803,)
Train Shape: (257487, 16) (257487,)


,msno,city,bd,registered_via,registration_init_time,registration_init_time_dt,total_order,payment_method_id_mode,payment_method_id_count,payment_plan_days_mode,payment_plan_days_mean,plan_list_price_mean,plan_lifetime_value,is_auto_renew_mode,transaction_date_max,cancel_times
703741,k5508nHWp+WmOeQTuChB4ETVmbhFmA+7E0qgK1uXKEg=,4,32,3,20130407,2013-04-07,22,33,1,30,21.82,108.36,2384.00,True,20170228,0
228194,WiRqDxpeBqgkDSqd3kRG/VgZbbDUh7yjxHquuY+iVbY=,13,33,3,20131125,2013-11-25,19,39,1,30,28.58,141.16,2682.00,True,20170131,0
735733,YVVOQPk0x2NeB3WqLgc9Sp1R8rIbaWDAnXAZFOf7rlY=,15,22,3,20150705,2015-07-05,17,40,3,30,30.00,158.12,2688.00,True,20170210,4
774256,pm07iNhaXPZ3On4cr4xyL4qOETYWikeemBOHZKhvzYI=,1,0,7,20140806,2014-08-06,26,41,1,30,28.85,143.27,3725.00,True,20170205,0
282188,MboyMUzNIj6LN+wDrZai6uLE3XQ0ReA79JSMNZfkLgk=,1,0,7,20121026,2012-10-26,27,41,1,30,28.89,132.00,3564.00,True,20170216,1


In [7]:
base_dataset = pd.concat([X_train, X_test])

In [10]:
base_dataset.drop(columns=['msno','registration_init_time','registration_init_time_dt'], axis=1).describe()

,bd,total_order,payment_method_id_mode,payment_method_id_count,payment_plan_days_mode,payment_plan_days_mean,plan_list_price_mean,plan_lifetime_value,transaction_date_max,cancel_times
count,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00
mean,13.43,15.88,38.80,1.20,32.08,32.17,129.32,2066.15,20169855.73,0.28
std,16.02,8.64,3.70,0.50,17.10,17.88,27.22,1230.96,1840.18,0.58
min,0.00,1.00,5.00,1.00,0.00,0.00,0.00,0.00,20150102.00,0.00
25%,0.00,8.00,38.00,1.00,30.00,28.85,99.00,990.00,20170205.00,0.00
50%,0.00,17.00,41.00,1.00,30.00,30.00,137.08,1937.00,20170214.00,0.00
75%,27.00,24.00,41.00,1.00,30.00,30.00,149.00,3174.00,20170223.00,0.00
max,90.00,64.00,41.00,8.00,240.00,240.00,252.00,8139.00,20170228.00,20.00


In [23]:
base_dataset.drop(columns=['msno','registration_init_time','registration_init_time_dt'], axis=1).describe(include='all')

,city,bd,registered_via,total_order,payment_method_id_mode,payment_method_id_count,payment_plan_days_mode,payment_plan_days_mean,plan_list_price_mean,plan_lifetime_value,is_auto_renew_mode,transaction_date_max,cancel_times
count,858290,858290.00,858290,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290.00,858290,858290.00,858290.00
unique,21,nan,5,nan,nan,nan,nan,nan,nan,nan,2,nan,nan
top,1,nan,7,nan,nan,nan,nan,nan,nan,nan,True,nan,nan
freq,441322,nan,462085,nan,nan,nan,nan,nan,nan,nan,736805,nan,nan
mean,NaN,13.43,NaN,15.88,38.80,1.20,32.08,32.17,129.32,2066.15,NaN,20169855.73,0.28
std,NaN,16.02,NaN,8.64,3.70,0.50,17.10,17.88,27.22,1230.96,NaN,1840.18,0.58
min,NaN,0.00,NaN,1.00,5.00,1.00,0.00,0.00,0.00,0.00,NaN,20150102.00,0.00
25%,NaN,0.00,NaN,8.00,38.00,1.00,30.00,28.85,99.00,990.00,NaN,20170205.00,0.00
50%,NaN,0.00,NaN,17.00,41.00,1.00,30.00,30.00,137.08,1937.00,NaN,20170214.00,0.00
75%,NaN,27.00,NaN,24.00,41.00,1.00,30.00,30.00,149.00,3174.00,NaN,20170223.00,0.00


In [19]:
base_dataset.head()

,msno,city,bd,registered_via,registration_init_time,registration_init_time_dt,total_order,payment_method_id_mode,payment_method_id_count,payment_plan_days_mode,payment_plan_days_mean,plan_list_price_mean,plan_lifetime_value,is_auto_renew_mode,transaction_date_max,cancel_times
703741,k5508nHWp+WmOeQTuChB4ETVmbhFmA+7E0qgK1uXKEg=,4,32,3,20130407,2013-04-07,22,33,1,30,21.82,108.36,2384.00,True,20170228,0
228194,WiRqDxpeBqgkDSqd3kRG/VgZbbDUh7yjxHquuY+iVbY=,13,33,3,20131125,2013-11-25,19,39,1,30,28.58,141.16,2682.00,True,20170131,0
735733,YVVOQPk0x2NeB3WqLgc9Sp1R8rIbaWDAnXAZFOf7rlY=,15,22,3,20150705,2015-07-05,17,40,3,30,30.00,158.12,2688.00,True,20170210,4
774256,pm07iNhaXPZ3On4cr4xyL4qOETYWikeemBOHZKhvzYI=,1,0,7,20140806,2014-08-06,26,41,1,30,28.85,143.27,3725.00,True,20170205,0
282188,MboyMUzNIj6LN+wDrZai6uLE3XQ0ReA79JSMNZfkLgk=,1,0,7,20121026,2012-10-26,27,41,1,30,28.89,132.00,3564.00,True,20170216,1


In [ ]:
## Set the list of the categorical columns in the dataset
cat_col = ['city', 'registered_via', 'payment_method_id_mode']

# Drop msno from the dataset
X_train.drop(['msno'], inplace=True, axis=1)
X_test.drop(['msno'], inplace=True, axis=1)

In [ ]:
RANDOM_STATE = 42 
CV_ITER = 5
SCORING_METRIC = 'recall'

BASE_NAME = "basic_" + str(CV_ITER) + "cv_" + SCORING_METRIC

## Each entry in the list is a Tuple of
##   [ModelName, Model, HyperParams, ScoringMetric]
classifiers = [
    ('NB_' + BASE_NAME, GaussianNB(), {}, SCORING_METRIC),
    ('DT_' + BASE_NAME, DecisionTreeClassifier(random_state=RANDOM_STATE), {}, SCORING_METRIC),
    ('RF_' + BASE_NAME, RandomForestClassifier(random_state=RANDOM_STATE, n_jobs =-1), {}, SCORING_METRIC),
    ('LR_' + BASE_NAME, LogisticRegression(random_state=RANDOM_STATE, solver = 'liblinear'), {}, SCORING_METRIC),
    ('MLP_' + BASE_NAME, MLPClassifier(random_state=RANDOM_STATE), {}, SCORING_METRIC),
    ('XGB_' + BASE_NAME, xgb.XGBClassifier(random_state=RANDOM_STATE, objective="binary:logistic"), {}, SCORING_METRIC) 
]

In [ ]:
all_metrics = pd.DataFrame()
all_results = list()

### Experiment 1.x Baseline - Default Settings - No sampling - 10 iterations to test score variance

In [ ]:
# %%time

# N_ITERATIONS = 10
# RANDOM_STATE = None

# sampler = ('None', DummySampler())

# exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, N_ITERATIONS, \
#                                                 cv_iter=CV_ITER, cat_col=cat_col)


### Is it possible to test RandomSampling and the effect of random_state ?????

In [ ]:
# %%time

# N_ITERATIONS = 10
# RANDOM_STATE = None

# sampler = ('None', RandomOverSampler(random_state=None))

# exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, N_ITERATIONS, \
#                                                 cv_iter=CV_ITER, cat_col=cat_col)


In [ ]:
#  Over 10 iterations we have 0 difference in accuracy, precision, recall, f1_score, aucroc, aucprc
#    Small difference in balanced accuracy of 0.01. 
#    A 5-fold CV recall score on the test set shows a mean of 0.45 with SD of 0.01.
#    Churn cost changes minimally (M: 8680470.0, SD: 6398.132540046353, Pct Cost of 1 SD: 0.07%))
    
#    np.mean(exp_metrics.model_churn_cost.values),  np.std(exp_metrics.model_churn_cost.values)
#    100 * (np.std(exp_metrics.model_churn_cost.values) / np.mean(exp_metrics.model_churn_cost.values))    

#exp_metrics

### Experiment 1.1 Baseline - Default Settings - No sampling

In [ ]:
%%time

sampler = ('None', DummySampler())

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, \
                                                cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.2 Baseline - Default Settings - Oversampled training set 100%

In [ ]:
%%time

sampler = ('ROS 1:1', RandomOverSampler(random_state=RANDOM_STATE))

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.3 Baseline - Default Settings - Undersampled training set

In [ ]:
%%time

sampler = ('RUS 1:1', RandomUnderSampler(random_state=RANDOM_STATE))

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.4 Baseline - Default Settings - Over sampling - 33% of majority size

In [ ]:
%%time

sampler = ('ROS 3:1', RandomOverSampler(random_state=RANDOM_STATE, sampling_strategy = 1/3))

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.5 Baseline - Default Settings - Over sampling - 66% of majority size

In [ ]:
%%time

sampler = ('ROS 3:2', RandomOverSampler(random_state=RANDOM_STATE, sampling_strategy = 2/3))

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.6 Baseline - Default Settings - Over sampling - 50% of majority size

In [ ]:
%%time

sampler = ('ROS 2:1', RandomOverSampler(random_state=RANDOM_STATE, sampling_strategy = 1/2))

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Experiment 1.7 Baseline Default Settings SMOTE-NC

In [ ]:
%%time

sampler = ('SMOTE_NC', SMOTENC(random_state=RANDOM_STATE, categorical_features=[0,1,2,3,4,5,6,7,12,13], n_jobs=8))

## Keep the following, essentially dropping the dt columns
cols = ['is_churn', 'city', 'bd', 'registered_via', 'total_order',
       'payment_method_id_mode', 'payment_method_id_count',
       'payment_plan_days_mode', 'payment_plan_days_mean',
       'plan_list_price_mean', 'plan_lifetime_value', 'actual_amount_mean',
       'total_actual_amount', 'is_auto_renew_mode', 'cancel_times']

exp_metrics, model_results = perform_experiment(X_train, X_test, y_train, y_test, classifiers, sampler, 1, 
                                 cv_iter=CV_ITER, cat_col=cat_col)

all_metrics = all_metrics.append(exp_metrics)
all_results.append((sampler[0]+"_"+BASE_NAME, exp_metrics, model_results))

### Persist results to disk

In [ ]:
store(all_results, 'Basic_DefaultHyper_{0}.pickle'.format(datetime.today().strftime('%Y-%m-%d')))

In [ ]:
import pickle

In [ ]:
with open('Basic_DefaultHyper_2019-08-02.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)
    


In [ ]:
data[3][1]

### Sort the results and get the best sampling config per classifier 

In [ ]:
top_results = all_metrics.sort_values(['balanced_accuracy'], ascending=[False]).groupby('classifier').head(1)
top_results

### Graph the results base on the list of best classifiers above

In [ ]:
temp = filter_top_model_results(top_results, all_results)
plot_roc_prc(temp)

### How long did each of these actually take ...

In [ ]:
all_metrics.sort_values(['auprc', 'recall'], ascending=[False, False]).head(15)

In [ ]:
ordered_features

In [ ]:
## Print the permutation importance 

import eli5
from eli5.sklearn import PermutationImportance

_, X_test, _, y_test = prepare_train_test_split(model_dataset, 0)

perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)

eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
#model_name, model = model_train_results[1][2]

for model_name, model in model_train_results[1]:
    probs = model.predict_proba(X_test)[:, 1]
    pr_data = plot_precision_recall(
        y_test, probs, title='PR Curve for {0}'.format(model_name))

In [ ]:
probs = model.predict_proba(X_test)[:, 1]
pr_data = plot_precision_recall(
    y_test, probs, title='PR Curve for {0}'.format(model_name))

In [ ]:
pr_data = plot_roc(
    y_test, probs, title='ROC Curve for {0}'.format(model_name))

In [ ]:
print(plt.style.available)

In [ ]:
model_dataset.head()

In [ ]:
print("Original Data distribution")
print(model_dataset['is_churn'].value_counts())

In [ ]:
# Display new class counts
print('Sci-Kit Learn : resample : Down Sampled data set')
train_downsample = undersampled_dataset(model_dataset, 'is_churn')

print(train_downsample['is_churn'].value_counts())
print("Num records = ", train_downsample.shape[0])
train_downsample.is_churn.value_counts().plot(kind='bar', title='Count (target)')

In [ ]:
# Display new class counts
print('Sci-Kit Learn : resample : Up Sampled data set')
train_upsample = oversampled_dataset(model_dataset, 'is_churn')

print(train_upsample['is_churn'].value_counts())
print("Num records = ", train_upsample.shape[0])
train_upsample.is_churn.value_counts().plot(kind='bar', title='Count (target)')

In [ ]:
probs = model.predict_proba(X_test)[:, 1]
pr_data = plot_precision_recall(
    y_test, probs, title='Precision-Recall Curve for Random Forest')

In [ ]:
precision_above = pr_data.loc[pr_data['precision'] >= 0.25].copy()
precision_above.sort_values('recall', ascending=False, inplace=True)
precision_above.head()

In [ ]:
threshold_required = 0.5
# Make predictions where probability is above threshold
preds = np.zeros(len(y_test))
preds[probs >= threshold_required] = 1

# Make and plot confusion matrix
cm = confusion_matrix(y_test, preds)
plot_confusion_matrix(cm, classes=['No Churn', 'Churn'],
                      title='Churn Confusion Matrix')

In [ ]:
model_dataset.head()

In [ ]:
fi = pd.DataFrame({'importance': model.feature_importances_}, index=model_dataset.iloc[:, 1:].columns).\
    sort_values('importance', ascending=False)
fi.head(10)